NETFLIX 의 영화/드라마데이터를 기반으로 새로운 영화/드라마를 추천해주는 서비스 (Content-based recommender)

데이터의 출처는 KAGGLE이며 한국 영화/드라마 데이터만 추출한 한국영화/드라마 추천 서비스

 **1. Importing libraries**



In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

**2. DATA Preprocessing**

In [2]:
netflix = pd.read_csv('netflix_titles.csv')

In [3]:
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


country 컬럼이 한국인것만 추출

In [4]:
netflix = netflix[netflix['country'] == 'South Korea']

In [5]:
netflix.isna().sum()

show_id           0
type              0
title             0
director        150
cast              3
country           0
date_added        0
release_year      0
rating            0
duration          0
listed_in         0
description       0
dtype: int64

director가 없는게 199중 150개라 director칼럼은 DROP

In [6]:
netflix = netflix.drop(columns='director',axis=1)

In [8]:
netflix.reset_index(inplace=True)
netflix.drop(columns='index',axis=1,inplace=True)

Content based recommendation system 이기 때문에 필요없는 칼럼들 제외

In [10]:
netflix = netflix.drop(columns=['show_id','cast','country','date_added','release_year','rating','duration'],axis=1)
netflix.head()

,type,title,listed_in,description
0,TV Show,Tayo the Little Bus,"Kids' TV, Korean TV Shows",As they learn their routes around the busy cit...
1,TV Show,Pororo - The Little Penguin,"Kids' TV, Korean TV Shows","On a tiny island, Pororo the penguin has fun a..."
2,TV Show,The Crowned Clown,"International TV Shows, Romantic TV Shows, TV ...","Standing in for an unhinged Joseon king, a loo..."
3,TV Show,Her Private Life,"International TV Shows, Romantic TV Shows, TV ...","An art curator's life unravels, as she tries t..."
4,TV Show,YooHoo to the Rescue,"Kids' TV, Korean TV Shows","In a series of magical missions, quick-witted ..."


**3. Vectorization & Recommendation Function**

필요라이브러리 추가 Import (sklearn과 regular expression 필요)

In [11]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

각 칼럼의 데이터들을 하나의 컬럼으로 합쳐 쉽게 가중치와 유사도를 계산하기 위한 전처리

In [12]:
netflix['listed_in'] = [re.sub(r'[^\w\s]','',t) for t in netflix['listed_in']]
netflix['description'] = [re.sub(r'[^\w\s]','',t) for t in netflix['description']]
netflix['listed_in'] = [t.lower() for t in netflix['listed_in']]
netflix['description'] = [t.lower() for t in netflix['description']]

netflix['combined'] = netflix['listed_in'] + ' ' + netflix['title'] + ' ' + netflix['description']
netflix

,type,title,listed_in,description,combined
0,TV Show,Tayo the Little Bus,kids tv korean tv shows,as they learn their routes around the busy cit...,kids tv korean tv shows Tayo the Little Bus as...
1,TV Show,Pororo - The Little Penguin,kids tv korean tv shows,on a tiny island pororo the penguin has fun ad...,kids tv korean tv shows Pororo - The Little Pe...
2,TV Show,The Crowned Clown,international tv shows romantic tv shows tv dr...,standing in for an unhinged joseon king a look...,international tv shows romantic tv shows tv dr...
3,TV Show,Her Private Life,international tv shows romantic tv shows tv co...,an art curators life unravels as she tries to ...,international tv shows romantic tv shows tv co...
4,TV Show,YooHoo to the Rescue,kids tv korean tv shows,in a series of magical missions quickwitted yo...,kids tv korean tv shows YooHoo to the Rescue i...
...,...,...,...,...,...
194,Movie,The President's Barber,comedies dramas international movies,the personal barber to the president as well a...,comedies dramas international movies The Presi...
195,Movie,The Prison,action adventure dramas international movies,a copturnedconvict discovers a crime syndicate...,action adventure dramas international movies ...
196,TV Show,This Is My Love,international tv shows korean tv shows romanti...,a renowned actor who is still pining over the ...,international tv shows korean tv shows romanti...
197,Movie,Train to Busan,action adventure horror movies international ...,as a zombie outbreak sweeps the country a dad ...,action adventure horror movies international ...


TfidVectorizer 와 linear_kernel 을 사용하여 가중치 매트릭스를 만들고 유사도를 구함 (linear_kernel 대시 cosine similarity를 사용해도 됨)

In [57]:
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(netflix['combined'])
similarity = linear_kernel(matrix, matrix)
title_index = pd.Series(netflix.index, index=netflix['title'])

유사도 

In [69]:
similarity

array([[1.        , 0.19236714, 0.04774552, ..., 0.04751899, 0.05978257,
        0.11749988],
       [0.19236714, 1.        , 0.08535582, ..., 0.05696698, 0.04175834,
        0.10399506],
       [0.04774552, 0.08535582, 1.        , ..., 0.07537726, 0.05114015,
        0.04877417],
       ...,
       [0.04751899, 0.05696698, 0.07537726, ..., 1.        , 0.08903124,
        0.06914164],
       [0.05978257, 0.04175834, 0.05114015, ..., 0.08903124, 1.        ,
        0.05072166],
       [0.11749988, 0.10399506, 0.04877417, ..., 0.06914164, 0.05072166,
        1.        ]])

Content를 추천해주는 함수 정의

In [70]:
def recommendation(title):
  idx = title_index[title]
  score = list(enumerate(similarity[idx]))
  score = sorted(score, key=lambda x:x[1], reverse=True)
  score = score[1:10]
  movie_title = [i[0] for i in score]
  return netflix['title'].iloc[movie_title]

결과

In [71]:
recommendation('#Alive')

44                   Goedam
197          Train to Busan
180    Memoir of a Murderer
92                 Vagabond
79         Tune in for Love
5             The 8th Night
149                 Pandora
140              Man to Man
126           Psychokinesis
Name: title, dtype: object

In [74]:
recommendation('Reply 1997')

52            Dear My Friends
165          12 Years Promise
47               Was It Love?
163    K-POP Extreme Survival
57                       Iris
26        A Love So Beautiful
66              Itaewon Class
3            Her Private Life
139               Magic Phone
Name: title, dtype: object